In [1]:
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install pandas_datareader

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr

# pd.set_option('display.max_columns', None) #show all columns
# pd.set_option('display.width', 1000) #prevent line wrapping

#show all columns and rows when printing -- rasthmi suggestion
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


pd.set_option("display.width", 0)
pd.set_option("display.max_colwidth", None)

# prevent notation -> better readability
pd.set_option("display.float_format", lambda x: f"{x:.2f}")


#date range for data
start_date = "1998-12-22"
end_date = "2025-04-02"

#tickers
tickers = {
    "IBM": "IBM",
    "XLK": "XLK",
    "SP500": "^GSPC",
    "VIX": "^VIX",
    "TNX": "^TNX",
    "Gold": "GC=F",
    "Oil": "CL=F",
    "USD_Index": "DX-Y.NYB"
}

#sector
sector_etfs = {
    "XLF": "XLF",
    "XLV": "XLV",
    "XLY": "XLY",
    "XLP": "XLP",
    "XLE": "XLE",
    "XLI": "XLI",
    "XLU": "XLU",
    "XLB": "XLB"
}


all_tickers_mapping = {**tickers, **sector_etfs}
download_list = list(all_tickers_mapping.values())

#download from yfinance
raw_data = yf.download(download_list, start=start_date, end=end_date, interval="1d", group_by="ticker", auto_adjust=False, threads=False)

# =store OHLCV data
ohlcv_data = {}
for custom_name, actual_ticker in all_tickers_mapping.items():
    try:
        df = raw_data[actual_ticker][["Open", "High", "Low", "Close", "Volume"]].copy()
        if not df.dropna().empty:
            df.columns = [f"{custom_name}_{col}" for col in df.columns]
            ohlcv_data[custom_name] = df
            print(f"Loaded OHLCV for {custom_name}")
        else:
            print(f"No data for {custom_name}")
    except Exception as e:
        print(f"Could not process {custom_name}: {e}")


merged_ohlcv = pd.concat(ohlcv_data.values(), axis=1)

# macro
fred_series = {
    "CPI": "CPIAUCNS",
    "Unemployment": "UNRATE"
}

for name, code in fred_series.items():
    try:
        print(f"Downloading FRED series: {name}...")
        fred_df = pdr.DataReader(code, "fred", start=start_date, end=end_date)
        fred_df = fred_df.resample("D").ffill()
        merged_ohlcv[name] = fred_df
    except Exception as e:
        print(f"Failed to fetch {name}: {e}")

#forward-fill and back-fill + clean up
merged_ohlcv = merged_ohlcv.ffill().bfill().loc[start_date:end_date]

print("\nFinal shape:", merged_ohlcv.shape)
memory_mb = merged_ohlcv.memory_usage(deep=True).sum() / (1024 ** 2)
print(f"Total memory usage: {memory_mb:.2f} MB")

merged_ohlcv.to_csv("final_macro_ohlcv_with_indicators.csv")


[*********************100%***********************]  16 of 16 completed


Loaded OHLCV for IBM
Loaded OHLCV for XLK
Loaded OHLCV for SP500
Loaded OHLCV for VIX
Loaded OHLCV for TNX
Loaded OHLCV for Gold
Loaded OHLCV for Oil
Loaded OHLCV for USD_Index
Loaded OHLCV for XLF
Loaded OHLCV for XLV
Loaded OHLCV for XLY
Loaded OHLCV for XLP
Loaded OHLCV for XLE
Loaded OHLCV for XLI
Loaded OHLCV for XLU
Loaded OHLCV for XLB

Final shape: (6649, 82)
Total memory usage: 4.21 MB


In [4]:
def keep_close_and_volume_columns(df: pd.DataFrame, tickers: dict) -> pd.DataFrame:
    #keep close and volume columns, get rid of multicollinearity
    columns_to_keep = []

    for ticker in tickers.keys():
        close_col = f"{ticker}_Close"
        volume_col = f"{ticker}_Volume"
        if close_col in df.columns:
            columns_to_keep.append(close_col)
        if volume_col in df.columns:
            columns_to_keep.append(volume_col)

    return df[columns_to_keep].copy()


cleaned_data = keep_close_and_volume_columns(merged_ohlcv, all_tickers_mapping)

In [5]:
cleaned_data.drop(columns=["VIX_Volume"], inplace=True)

In [6]:
cleaned_data.columns

Index(['IBM_Close', 'IBM_Volume', 'XLK_Close', 'XLK_Volume', 'SP500_Close',
       'SP500_Volume', 'VIX_Close', 'TNX_Close', 'TNX_Volume', 'Gold_Close',
       'Gold_Volume', 'Oil_Close', 'Oil_Volume', 'USD_Index_Close',
       'USD_Index_Volume', 'XLF_Close', 'XLF_Volume', 'XLV_Close',
       'XLV_Volume', 'XLY_Close', 'XLY_Volume', 'XLP_Close', 'XLP_Volume',
       'XLE_Close', 'XLE_Volume', 'XLI_Close', 'XLI_Volume', 'XLU_Close',
       'XLU_Volume', 'XLB_Close', 'XLB_Volume'],
      dtype='object')

In [7]:
import numpy as np

#CHATGPT helped trmeendously with this function, in terms of providing general structure and also giving us the equations for some of the technical indicators
def compute_technical_indicators(df, prefix):
    #function to help us create OHCLV and technical indicators for IBM stock

    #extract price and volume series
    #OHLCV data
    open_price = df[f"{prefix}_Open"]
    high = df[f"{prefix}_High"]
    low = df[f"{prefix}_Low"]
    close = df[f"{prefix}_Close"]
    volume = df[f"{prefix}_Volume"]

    features = pd.DataFrame(index=df.index)

    #returns
    #daily, 5 day, 10 day log returns + rolling mean (20-day)
    features[f"{prefix}_LogReturn"] = np.log(close / close.shift(1))
    features[f"{prefix}_Return_5d"] = close / close.shift(5) - 1
    features[f"{prefix}_Return_10d"] = close / close.shift(10) - 1
    features[f"{prefix}_RollingMean_Return_20d"] = features[f"{prefix}_LogReturn"].rolling(20).mean()

    #rsi 14 day of IBM stock -- equation from yahoo finance
    delta = close.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    features[f"{prefix}_RSI"] = 100 - (100 / (1 + rs))

    #signal (MACD)  - technical indicator to help us analyze momentum in the IBM price
    ema12 = close.ewm(span=12, adjust=False).mean()
    ema26 = close.ewm(span=26, adjust=False).mean()
    macd = ema12 - ema26
    signal = macd.ewm(span=9, adjust=False).mean()
    features[f"{prefix}_MACD"] = macd
    features[f"{prefix}_MACD_Signal"] = signal
    features[f"{prefix}_MACD_Hist"] = macd - signal

    #ROC 12 day
    features[f"{prefix}_ROC_12d"] = (close / close.shift(12) - 1) * 100

    # volitility
    features[f"{prefix}_Rolling_STD_20d"] = close.rolling(20).std()

    # avg true range 14 day
    tr1 = high - low
    tr2 = (high - close.shift(1)).abs()
    tr3 = (low - close.shift(1)).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    features[f"{prefix}_ATR_14d"] = tr.rolling(14).mean()

    # bollinger bands (20-day): SMA 20
    sma20 = close.rolling(20).mean()
    std20 = close.rolling(20).std()
    features[f"{prefix}_Bollinger_Upper"] = sma20 + 2 * std20
    features[f"{prefix}_Bollinger_Lower"] = sma20 - 2 * std20
    features[f"{prefix}_Bollinger_Mid"] = sma20

    # simple moving average, and EMA both 20 and 50 day
    features[f"{prefix}_SMA_20"] = close.rolling(20).mean()
    features[f"{prefix}_SMA_50"] = close.rolling(50).mean()
    features[f"{prefix}_EMA_20"] = close.ewm(span=20, adjust=False).mean()
    features[f"{prefix}_EMA_50"] = close.ewm(span=50, adjust=False).mean()

    # on balance volume OBV
    direction = np.where(close > close.shift(1), 1, np.where(close < close.shift(1), -1, 0))
    obv = (direction * volume).cumsum()
    features[f"{prefix}_OBV"] = obv

    # Money Flow Index -- help us determine actual cash flow in and out of IBM stock
    typical_price = (high + low + close) / 3
    money_flow = typical_price * volume
    pos_flow = money_flow.where(typical_price > typical_price.shift(1), 0)
    neg_flow = money_flow.where(typical_price < typical_price.shift(1), 0)
    pos_mf = pos_flow.rolling(14).sum()
    neg_mf = neg_flow.rolling(14).sum()
    features[f"{prefix}_MFI"] = 100 - (100 / (1 + pos_mf / neg_mf))

    return features

In [8]:
#compute technical indicators for each ticker
all_features = {}
for ticker in all_tickers_mapping.keys():
    required_cols = [f"{ticker}_{c}" for c in ["Open", "High", "Low", "Close", "Volume"]]
    if all(col in merged_ohlcv.columns for col in required_cols):
        df_ticker = merged_ohlcv[required_cols]
        features = compute_technical_indicators(df_ticker, ticker)
        all_features[ticker] = features
    else:
        print(f"Skipping indicators for {ticker}: missing required OHLCV columns.")

# merge raw data with engineered features
features_df = pd.concat(all_features.values(), axis=1)
full_df = pd.concat([merged_ohlcv, features_df], axis=1)
print("full_df shape before cleaning:", full_df.shape)

# drop features with >50% missing values
missing_counts = full_df.isna().sum()
threshold = 0.5 * len(full_df)
cols_to_drop = missing_counts[missing_counts > threshold].index.tolist()
print("Dropping features with >50% missing values:", cols_to_drop)
full_df = full_df.drop(columns=cols_to_drop)

# drop any rows that still have NaNs
full_df = full_df.dropna()
print("Shape after dropna:", full_df.shape)

# drop redundant raw OHLC columns and constant zero volume columns
raw_to_drop = []
for ticker in all_tickers_mapping.keys():
    raw_to_drop.extend([f"{ticker}_Open", f"{ticker}_High", f"{ticker}_Low"])
#drop VIX_Volume if present and constant
if "VIX_Volume" in full_df.columns:
    raw_to_drop.append("VIX_Volume")

#keep only those that exist
raw_to_drop = [col for col in raw_to_drop if col in full_df.columns]
cleaned_data = full_df.drop(columns=raw_to_drop)

print("Cleaned data shape:", cleaned_data.shape)
print("Cleaned data columns:\n", cleaned_data.columns.tolist())


full_df shape before cleaning: (6649, 402)
Dropping features with >50% missing values: ['VIX_MFI', 'TNX_MFI', 'USD_Index_MFI']
Shape after dropna: (6180, 399)
Cleaned data shape: (6180, 350)
Cleaned data columns:
 ['IBM_Close', 'IBM_Volume', 'XLK_Close', 'XLK_Volume', 'SP500_Close', 'SP500_Volume', 'VIX_Close', 'TNX_Close', 'TNX_Volume', 'Gold_Close', 'Gold_Volume', 'Oil_Close', 'Oil_Volume', 'USD_Index_Close', 'USD_Index_Volume', 'XLF_Close', 'XLF_Volume', 'XLV_Close', 'XLV_Volume', 'XLY_Close', 'XLY_Volume', 'XLP_Close', 'XLP_Volume', 'XLE_Close', 'XLE_Volume', 'XLI_Close', 'XLI_Volume', 'XLU_Close', 'XLU_Volume', 'XLB_Close', 'XLB_Volume', 'CPI', 'Unemployment', 'IBM_LogReturn', 'IBM_Return_5d', 'IBM_Return_10d', 'IBM_RollingMean_Return_20d', 'IBM_RSI', 'IBM_MACD', 'IBM_MACD_Signal', 'IBM_MACD_Hist', 'IBM_ROC_12d', 'IBM_Rolling_STD_20d', 'IBM_ATR_14d', 'IBM_Bollinger_Upper', 'IBM_Bollinger_Lower', 'IBM_Bollinger_Mid', 'IBM_SMA_20', 'IBM_SMA_50', 'IBM_EMA_20', 'IBM_EMA_50', 'IBM_OBV'

/shared/courseSharedFolders/142601outer/142601/cs109b/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
cleaned_data.describe()

,IBM_Close,IBM_Volume,XLK_Close,XLK_Volume,SP500_Close,SP500_Volume,VIX_Close,TNX_Close,TNX_Volume,Gold_Close,Gold_Volume,Oil_Close,Oil_Volume,USD_Index_Close,USD_Index_Volume,XLF_Close,XLF_Volume,XLV_Close,XLV_Volume,XLY_Close,XLY_Volume,XLP_Close,XLP_Volume,XLE_Close,XLE_Volume,XLI_Close,XLI_Volume,XLU_Close,XLU_Volume,XLB_Close,XLB_Volume,CPI,Unemployment,IBM_LogReturn,IBM_Return_5d,IBM_Return_10d,IBM_RollingMean_Return_20d,IBM_RSI,IBM_MACD,IBM_MACD_Signal,IBM_MACD_Hist,IBM_ROC_12d,IBM_Rolling_STD_20d,IBM_ATR_14d,IBM_Bollinger_Upper,IBM_Bollinger_Lower,IBM_Bollinger_Mid,IBM_SMA_20,IBM_SMA_50,IBM_EMA_20,IBM_EMA_50,IBM_OBV,IBM_MFI,XLK_LogReturn,XLK_Return_5d,XLK_Return_10d,XLK_RollingMean_Return_20d,XLK_RSI,XLK_MACD,XLK_MACD_Signal,XLK_MACD_Hist,XLK_ROC_12d,XLK_Rolling_STD_20d,XLK_ATR_14d,XLK_Bollinger_Upper,XLK_Bollinger_Lower,XLK_Bollinger_Mid,XLK_SMA_20,XLK_SMA_50,XLK_EMA_20,XLK_EMA_50,XLK_OBV,XLK_MFI,SP500_LogReturn,SP500_Return_5d,SP500_Return_10d,SP500_RollingMean_Return_20d,SP500_RSI,SP500_MACD,SP500_MACD_Signal,SP500_MACD_Hist,SP500_ROC_12d,SP500_Rolling_STD_20d,SP500_ATR_14d,SP500_Bollinger_Upper,SP500_Bollinger_Lower,SP500_Bollinger_Mid,SP500_SMA_20,SP500_SMA_50,SP500_EMA_20,SP500_EMA_50,SP500_OBV,SP500_MFI,VIX_LogReturn,VIX_Return_5d,VIX_Return_10d,VIX_RollingMean_Return_20d,VIX_RSI,VIX_MACD,VIX_MACD_Signal,VIX_MACD_Hist,VIX_ROC_12d,VIX_Rolling_STD_20d,VIX_ATR_14d,VIX_Bollinger_Upper,VIX_Bollinger_Lower,VIX_Bollinger_Mid,VIX_SMA_20,VIX_SMA_50,VIX_EMA_20,VIX_EMA_50,VIX_OBV,TNX_LogReturn,TNX_Return_5d,TNX_Return_10d,TNX_RollingMean_Return_20d,TNX_RSI,TNX_MACD,TNX_MACD_Signal,TNX_MACD_Hist,TNX_ROC_12d,TNX_Rolling_STD_20d,TNX_ATR_14d,TNX_Bollinger_Upper,TNX_Bollinger_Lower,TNX_Bollinger_Mid,TNX_SMA_20,TNX_SMA_50,TNX_EMA_20,TNX_EMA_50,TNX_OBV,Gold_LogReturn,Gold_Return_5d,Gold_Return_10d,Gold_RollingMean_Return_20d,Gold_RSI,Gold_MACD,Gold_MACD_Signal,Gold_MACD_Hist,Gold_ROC_12d,Gold_Rolling_STD_20d,Gold_ATR_14d,Gold_Bollinger_Upper,Gold_Bollinger_Lower,Gold_Bollinger_Mid,Gold_SMA_20,Gold_SMA_50,Gold_EMA_20,Gold_EMA_50,Gold_OBV,Gold_MFI,Oil_LogReturn,Oil_Return_5d,Oil_Return_10d,Oil_RollingMean_Return_20d,Oil_RSI,Oil_MACD,Oil_MACD_Signal,Oil_MACD_Hist,Oil_ROC_12d,Oil_Rolling_STD_20d,Oil_ATR_14d,Oil_Bollinger_Upper,Oil_Bollinger_Lower,Oil_Bollinger_Mid,Oil_SMA_20,Oil_SMA_50,Oil_EMA_20,Oil_EMA_50,Oil_OBV,Oil_MFI,USD_Index_LogReturn,USD_Index_Return_5d,USD_Index_Return_10d,USD_Index_RollingMean_Return_20d,USD_Index_RSI,USD_Index_MACD,USD_Index_MACD_Signal,USD_Index_MACD_Hist,USD_Index_ROC_12d,USD_Index_Rolling_STD_20d,USD_Index_ATR_14d,USD_Index_Bollinger_Upper,USD_Index_Bollinger_Lower,USD_Index_Bollinger_Mid,USD_Index_SMA_20,USD_Index_SMA_50,USD_Index_EMA_20,USD_Index_EMA_50,USD_Index_OBV,XLF_LogReturn,XLF_Return_5d,XLF_Return_10d,XLF_RollingMean_Return_20d,XLF_RSI,XLF_MACD,XLF_MACD_Signal,XLF_MACD_Hist,XLF_ROC_12d,XLF_Rolling_STD_20d,XLF_ATR_14d,XLF_Bollinger_Upper,XLF_Bollinger_Lower,XLF_Bollinger_Mid,XLF_SMA_20,XLF_SMA_50,XLF_EMA_20,XLF_EMA_50,XLF_OBV,XLF_MFI,XLV_LogReturn,XLV_Return_5d,XLV_Return_10d,XLV_RollingMean_Return_20d,XLV_RSI,XLV_MACD,XLV_MACD_Signal,XLV_MACD_Hist,XLV_ROC_12d,XLV_Rolling_STD_20d,XLV_ATR_14d,XLV_Bollinger_Upper,XLV_Bollinger_Lower,XLV_Bollinger_Mid,XLV_SMA_20,XLV_SMA_50,XLV_EMA_20,XLV_EMA_50,XLV_OBV,XLV_MFI,XLY_LogReturn,XLY_Return_5d,XLY_Return_10d,XLY_RollingMean_Return_20d,XLY_RSI,XLY_MACD,XLY_MACD_Signal,XLY_MACD_Hist,XLY_ROC_12d,XLY_Rolling_STD_20d,XLY_ATR_14d,XLY_Bollinger_Upper,XLY_Bollinger_Lower,XLY_Bollinger_Mid,XLY_SMA_20,XLY_SMA_50,XLY_EMA_20,XLY_EMA_50,XLY_OBV,XLY_MFI,XLP_LogReturn,XLP_Return_5d,XLP_Return_10d,XLP_RollingMean_Return_20d,XLP_RSI,XLP_MACD,XLP_MACD_Signal,XLP_MACD_Hist,XLP_ROC_12d,XLP_Rolling_STD_20d,XLP_ATR_14d,XLP_Bollinger_Upper,XLP_Bollinger_Lower,XLP_Bollinger_Mid,XLP_SMA_20,XLP_SMA_50,XLP_EMA_20,XLP_EMA_50,XLP_OBV,XLP_MFI,XLE_LogReturn,XLE_Return_5d,XLE_Return_10d,XLE_RollingMean_Return_20d,XLE_RSI,XLE_MACD,XLE_MACD_Signal,XLE_MACD_Hist,XLE_ROC_12d,XLE_Rolling_

In [10]:
cleaned_data.shape

(6180, 350)

In [11]:
log_columns = [
    "IBM_Volume",
    "XLK_Close",
    "XLK_Volume",
    "SP500_Close",
    "SP500_Volume",
    "VIX_Close",
    "Gold_Volume",
    "Oil_Volume",
    "XLF_Volume",
    "XLV_Close",
    "XLV_Volume",
    "XLY_Close",
    "XLY_Volume",
    "XLP_Close",
    "XLP_Volume",
    "XLE_Volume",
    "XLI_Volume",
    "XLU_Volume",
    "XLB_Volume",
    "IBM_Rolling_STD_20d",
    "IBM_ATR_14d",
    "XLK_Rolling_STD_20d",
    "XLK_ATR_14d",
    "XLK_Bollinger_Upper",
    "XLK_Bollinger_Lower",
    "XLK_Bollinger_Mid",
    "XLK_SMA_20",
    "XLK_SMA_50",
    "XLK_SMA_50",
    "XLK_EMA_20",
    "XLK_EMA_50",
    "SP500_Rolling_STD_20d",
    "SP500_ATR_14d",
    "SP500_Bollinger_Upper",
    "SP500_Bollinger_Lower",
    "SP500_Bollinger_Mid",
    "SP500_SMA_20",
    "SP500_SMA_50",
    "SP500_EMA_20",
    "SP500_EMA_50"
    "VIX_Rolling_STD_20d",
    "VIX_ATR_14d",
    "VIX_Bollinger_Upper",
    "VIX_Bollinger_Lower",
    "VIX_Bollinger_Mid",
    "VIX_SMA_20",
    "VIX_SMA_50",
    "VIX_EMA_20",
    "VIX_EMA_50",
    "TNX_Rolling_STD_20d",
    "TNX_ATR_14d",
    "Gold_Rolling_STD_20d",
    "Gold_atr_14d",
    "Oil_Rolling_STD_20d",
    "Oil_ATR_14d",
    "USD_Index_Rolling_std_20d",
    "USD_Index_ATR_14d",
    "XLF_Rolling_STD_20d",
    "XLF_ATR_24d",
    "XLV_Rolling_STD_20d",
    "XLV_ATR_14d",
    "XLY_Rolling_STD_20d",
    "XLY_ATR_14d",
    "XLY_Bollinger_Upper",
    "XLY_Bollinger_Lower",
    "XLY_Bollinger_Mid",
    "XLY_SMA_20",
    "XLY_SMA_50",
    "XLY_EMA_20",
    "XLY_EMA_50",
    "XLY_OBV",
    "XLP_ATR_14d",
    "XLP_Rolling_STD_20d",
    "XLP_ATR_14d",
    "XLP_Bollinger_Upper",
    "XLP_Bollinger_Lower",
    "XLP_Bollinger_Mid",
    "XLP_SMA_20",
    "XLP_SMA_50",
    "XLP_EMA_20",
    "XLP_EMA_50",
    "XLP_OBV",
    "XLE_Rolling_STD_20d",
    "XLE_ATR_14d",
    "XLI_Rolling_STD_20d",
    "XLI_ATR_14d",
    "XLI_Bollinger_Upper",
    "XLI_Bollinger_Lower",
    "XLI_Bollinger_Mid",
    "XLI_SMA_20",
    "XLI_SMA_50",
    "XLI_EMA_20",
    "XLI_SEA_50",
    "XLB_Rolling_STD_20d",
    "XLB_ATR_14d",
]

In [12]:
from sklearn.preprocessing import StandardScaler

def preprocess_features_with_log_and_standardization(
    data: pd.DataFrame,
    columns_to_log_transform: list[str],
    target_column_name: str = "IBM_Return_5d"
) -> pd.DataFrame:
    data_transformed = data.copy()

    for column_name in columns_to_log_transform:
        if column_name in data_transformed.columns:
            data_transformed[column_name] = np.log1p(data_transformed[column_name])

    numeric_feature_columns = [
        column for column in data_transformed.columns
        if column != target_column_name and np.issubdtype(data_transformed[column].dtype, np.number)
    ]

    scaler = StandardScaler()
    data_transformed[numeric_feature_columns] = scaler.fit_transform(data_transformed[numeric_feature_columns])

    return data_transformed

In [13]:
processed_data = preprocess_features_with_log_and_standardization(
    data=cleaned_data,
    columns_to_log_transform=log_columns,
    target_column_name="IBM_Return_5d"
)

/shared/courseSharedFolders/142601outer/142601/cs109b/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [14]:
processed_data.drop(columns=["XLY_OBV", "XLP_OBV"], inplace=True)

In [15]:
processed_data["IBM_Close_raw"] = merged_ohlcv["IBM_Close"]

/tmp/ipykernel_29260/2605016621.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  processed_data["IBM_Close_raw"] = merged_ohlcv["IBM_Close"]


In [16]:
# Save to CSV
# processed_data.to_csv("processed_data.csv", index=False)

processed_data.to_csv("processed_data.csv", index = False)